## Paquetes y datos originales ## 

In [1483]:
import json
import pandas as pd
from datetime import datetime
from listas import disfagia_list, PA_list, P_list, Main_respiratory_infections_list, LRTI_list, COPD_exacerbations_list, Pulmonary_fibrosis_fibrotorax_list
from auxiliary_functions import valores_codigos, contar_diccionarios, dias_ingreso_total, asignar_intervalo_edad, sumar_barthel, sumar_emina, obtener_ultimo_resultat, obtener_valor_promedio, canadenca_comparada, disfagia_mecvvs, extraer_valor_clave, extraer_valor_clave_simple

tabla = pd.read_pickle('./data/processed/dataframe.pkl')
import scipy.stats as stats
from scipy.stats import ttest_ind
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.proportion import proportions_ztest

### Previamente he tratado la base de datos, creando nuevas columnas para facilitar la lectura de los datos

In [1484]:
tabla.head()

,edat,sexe,visitaDietista,labs,mna,emina,canadenca,pes,atcs,mecvvs,...,albumina,proteinas totales,Hb,colesterol total,leucos,limfos,prot C react,urea,FGE MDRD,FGE CDK-EPI
0,89,F,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'M01AE', 'dataInici': '2020-06-21 00...",[],...,None,None,None,None,None,None,None,None,None,None
1,70,M,0,"[{'date': '20220524', 'name': 'PCR SARS-CoV2 S...",[],"[{'activitat': '0', 'estatMental': '0', 'humit...",[],"[{'valor': '64.7', 'data': '2022-10-04'}, {'va...","[{'codi': 'C09AA', 'dataInici': '2021-05-26 00...","[{'data': '20220930', 'pacientNoColabora': '',...",...,3.1,5.3,14.5,124.7,X,38.9,19.61,25.7,>60,>90
2,96,M,0,[],[],"[{'activitat': '3', 'estatMental': '0', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-09 00...",[],...,None,None,None,None,None,None,None,None,None,None
3,85,M,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-15 00...","[{'data': '20201204131810', 'disfagia': 'S', '...",...,None,None,None,None,None,None,None,None,None,None
4,93,M,0,"[{'date': '20220208', 'name': 'UREA Sèrum', 'l...","[{'imc': '0', 'anorexia': '1', 'perduaPes': '1...","[{'activitat': '2', 'estatMental': '1', 'humit...",[],"[{'valor': '49.900', 'data': '2022-06-02'}, {'...","[{'codi': 'N02AA', 'dataInici': '2022-02-01 00...","[{'data': '20220330172303', 'disfagiaConeguda'...",...,Proves afegides a un altre petició per extracc...,Proves afegides a un altre petició per extracc...,X,None,X,X,0.54,Proves afegides a un altre petició per extracc...,Proves afegides a un altre petició per extracc...,Proves afegides a un altre petició per extracc...


## Dividiré los datos en 3 grupos:
- Con PA (codigo ICD de PA )
- Con PA + MECVV (MECVV positivo + icd pneumonia < 30 days)
- Sin PA (ICD pneumonia or icd pneumonia + mecvv positive > 30 days)

# CON PA 

In [1485]:
tabla.PA_diagnosticada.value_counts()

PA_diagnosticada
0.0    33
1.0    17
Name: count, dtype: int64

In [1486]:
CON_PA = tabla[tabla["PA_diagnosticada"] == 1.0]
CON_PA.head(len(tabla))

,edat,sexe,visitaDietista,labs,mna,emina,canadenca,pes,atcs,mecvvs,...,albumina,proteinas totales,Hb,colesterol total,leucos,limfos,prot C react,urea,FGE MDRD,FGE CDK-EPI
0,89,F,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'M01AE', 'dataInici': '2020-06-21 00...",[],...,None,None,None,None,None,None,None,None,None,None
1,70,M,0,"[{'date': '20220524', 'name': 'PCR SARS-CoV2 S...",[],"[{'activitat': '0', 'estatMental': '0', 'humit...",[],"[{'valor': '64.7', 'data': '2022-10-04'}, {'va...","[{'codi': 'C09AA', 'dataInici': '2021-05-26 00...","[{'data': '20220930', 'pacientNoColabora': '',...",...,3.1,5.3,14.5,124.7,X,38.9,19.61,25.7,>60,>90
3,85,M,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-15 00...","[{'data': '20201204131810', 'disfagia': 'S', '...",...,None,None,None,None,None,None,None,None,None,None
6,101,F,0,[],[],"[{'activitat': '3', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'C09AA', 'dataInici': '2021-08-21 00...","[{'data': '20210819124116', 'disfagia': 'S', '...",...,None,None,None,None,None,None,None,None,None,None
12,86,M,0,[],[],"[{'activitat': '1', 'estatMental': '1', 'humit...",[],"[{'valor': '46.4', 'data': '2020-09-09'}]","[{'codi': 'G04CA', 'dataInici': '2020-09-04 00...","[{'data': '20200907110400', 'disfagia': 'S', '...",...,None,None,None,None,None,None,None,None,None,None
13,71,F,0,"[{'date': '20220216', 'name': 'CULTIU Esput', ...",[],"[{'activitat': '3', 'estatMental': '0', 'humit...","[{'dataValoracio': '2020-09-06', 'horaValoraci...",[],"[{'codi': 'C10AA', 'dataInici': '2020-09-06 00...","[{'data': '20200906175506', 'disfagia': 'N', '...",...,4.0,7.4,14.4,80.5,13.95,14.5,6.73,66.0,>60,70
15,75,M,0,[],[],"[{'activitat': '3', 'estatMental': '2', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-09-19 00...",[],...,None,None,None,None,None,None,None,None,None,None
26,97,F,0,"[{'date': '20220419', 'name': 'GLUCOSA Sèrum',...",[],"[{'activitat': '1', 'estatMental': '0', 'humit...",[],[],"[{'codi': 'C09BA', 'dataInici': '2022-04-17 00...",[],...,3.2,6.5,12.0,156.9,14.71,5.0,11.37,119.7,52,48
28,95,F,0,"[{'date': '20220526', 'name': 'GLUCOSA Sèrum',...",[],"[{'activitat': '3', 'estatMental': '0', 'humit...",[],"[{'valor': '0', 'data': '2022-05-30'}]","[{'codi': 'C10AA', 'dataInici': '2020-11-18 00...","[{'data': '20220529115841', 'disfagia': 'S', '...",...,3.5,5.8,8.8,186.4,8.31,11.7,2.33,49.9,51,48
29,97,F,0,[],[],"[{'activitat': '3', 'estatMental': '2', 'humit...",[],[],"[{'codi': 'N02BE', 'dataInici': '2020-11-30 00...",[],...,None,None,None,None,None,None,None,None,None,None


# CON_PA + MECVV

In [1487]:
CON_PA_MECVV = tabla[(tabla["P_diagnosticada"] == 1.0) & (tabla["Disfagia_mecvvs"] == 1.0) & (tabla["Dias_totales_ingresado"] < 30)]
CON_PA_MECVV.head(len(tabla))

,edat,sexe,visitaDietista,labs,mna,emina,canadenca,pes,atcs,mecvvs,...,albumina,proteinas totales,Hb,colesterol total,leucos,limfos,prot C react,urea,FGE MDRD,FGE CDK-EPI
3,85,M,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-15 00...","[{'data': '20201204131810', 'disfagia': 'S', '...",...,None,None,None,None,None,None,None,None,None,None
6,101,F,0,[],[],"[{'activitat': '3', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'C09AA', 'dataInici': '2021-08-21 00...","[{'data': '20210819124116', 'disfagia': 'S', '...",...,None,None,None,None,None,None,None,None,None,None
12,86,M,0,[],[],"[{'activitat': '1', 'estatMental': '1', 'humit...",[],"[{'valor': '46.4', 'data': '2020-09-09'}]","[{'codi': 'G04CA', 'dataInici': '2020-09-04 00...","[{'data': '20200907110400', 'disfagia': 'S', '...",...,None,None,None,None,None,None,None,None,None,None
13,71,F,0,"[{'date': '20220216', 'name': 'CULTIU Esput', ...",[],"[{'activitat': '3', 'estatMental': '0', 'humit...","[{'dataValoracio': '2020-09-06', 'horaValoraci...",[],"[{'codi': 'C10AA', 'dataInici': '2020-09-06 00...","[{'data': '20200906175506', 'disfagia': 'N', '...",...,4.0,7.4,14.4,80.5,13.95,14.5,6.73,66.0,>60,70
27,100,M,0,[],[],"[{'activitat': '3', 'estatMental': '0', 'humit...",[],"[{'valor': '0', 'data': '2021-09-14'}, {'valor...","[{'codi': 'C08CA', 'dataInici': '2020-11-23 00...","[{'data': '20201125131827', 'disfagia': 'S', '...",...,None,None,None,None,None,None,None,None,None,None
37,86,M,0,"[{'date': '20221003', 'name': 'GLUCOSA Sèrum',...","[{'imc': '', 'anorexia': '', 'perduaPes': '', ...","[{'activitat': '2', 'estatMental': '1', 'humit...",[],"[{'valor': '72', 'data': '2022-10-09'}]","[{'codi': 'G04CA', 'dataInici': '2022-09-26 00...","[{'data': '20221012122425', 'disfagia': 'S', '...",...,2.9,5.6,13.5,119.5,8.55,9.5,8.16,114.2,None,46
39,87,F,0,"[{'date': '20230619', 'name': 'GLUCOSA Sèrum',...",[],"[{'activitat': '1', 'estatMental': '0', 'humit...",[],[],"[{'codi': 'N02BE', 'dataInici': '2021-01-17 00...","[{'data': '20220111085147', 'disfagia': 'S', '...",...,None,6.2,11.2,None,22.39,X,14.21,30.7,None,>90
41,93,F,0,"[{'date': '20221013', 'name': 'GLUCOSA Sèrum',...",[],"[{'activitat': '1', 'estatMental': '1', 'humit...",[],"[{'valor': '0', 'data': '2023-06-02'}, {'valor...","[{'codi': 'C10AA', 'dataInici': '2021-02-05 00...","[{'data': '20230530155654', 'disfagia': 'S', '...",...,3.5,6.7,13.8,135.7,7.42,27.6,6.43,38.6,>60,48
46,86,M,0,[],[],"[{'activitat': '3', 'estatMental': '1', 'humit...",[],"[{'valor': '55.1', 'data': '2021-03-21'}]","[{'codi': 'G04CA', 'dataInici': '2021-03-20 00...","[{'data': '20210721140136', 'disfagia': 'S', '...",...,None,None,None,None,None,None,None,None,None,None
47,94,M,0,[],[],"[{'activitat': '3', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'C09AA', 'dataInici': '2021-03-23 00...","[{'data': '20210325091551', 'disfagia': 'S', '...",...,None,None,None,None,None,None,None,None,None,None


# SIN_PA

In [1488]:
SIN_PA = tabla[(tabla["P_diagnosticada"] == 1.0) & (tabla["Disfagia_mecvvs"] == 1.0) & (tabla["Dias_totales_ingresado"] > 30) | (tabla["P_diagnosticada"] == 1.0)]
SIN_PA.head(len(tabla))

,edat,sexe,visitaDietista,labs,mna,emina,canadenca,pes,atcs,mecvvs,...,albumina,proteinas totales,Hb,colesterol total,leucos,limfos,prot C react,urea,FGE MDRD,FGE CDK-EPI
0,89,F,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'M01AE', 'dataInici': '2020-06-21 00...",[],...,None,None,None,None,None,None,None,None,None,None
1,70,M,0,"[{'date': '20220524', 'name': 'PCR SARS-CoV2 S...",[],"[{'activitat': '0', 'estatMental': '0', 'humit...",[],"[{'valor': '64.7', 'data': '2022-10-04'}, {'va...","[{'codi': 'C09AA', 'dataInici': '2021-05-26 00...","[{'data': '20220930', 'pacientNoColabora': '',...",...,3.1,5.3,14.5,124.7,X,38.9,19.61,25.7,>60,>90
2,96,M,0,[],[],"[{'activitat': '3', 'estatMental': '0', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-09 00...",[],...,None,None,None,None,None,None,None,None,None,None
3,85,M,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-15 00...","[{'data': '20201204131810', 'disfagia': 'S', '...",...,None,None,None,None,None,None,None,None,None,None
4,93,M,0,"[{'date': '20220208', 'name': 'UREA Sèrum', 'l...","[{'imc': '0', 'anorexia': '1', 'perduaPes': '1...","[{'activitat': '2', 'estatMental': '1', 'humit...",[],"[{'valor': '49.900', 'data': '2022-06-02'}, {'...","[{'codi': 'N02AA', 'dataInici': '2022-02-01 00...","[{'data': '20220330172303', 'disfagiaConeguda'...",...,Proves afegides a un altre petició per extracc...,Proves afegides a un altre petició per extracc...,X,None,X,X,0.54,Proves afegides a un altre petició per extracc...,Proves afegides a un altre petició per extracc...,Proves afegides a un altre petició per extracc...
5,57,M,0,"[{'date': '20220909', 'name': 'GLUCOSA Sèrum',...",[],"[{'activitat': '1', 'estatMental': '0', 'humit...",[],"[{'valor': '83.7', 'data': '2020-08-13'}, {'va...","[{'codi': 'N02BE', 'dataInici': '2020-08-12 00...",[],...,4.7,6.3,14.9,186,9.45,33.0,None,30.4,>60,>90
6,101,F,0,[],[],"[{'activitat': '3', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'C09AA', 'dataInici': '2021-08-21 00...","[{'data': '20210819124116', 'disfagia': 'S', '...",...,None,None,None,None,None,None,None,None,None,None
7,90,F,0,"[{'date': '20220627', 'name': 'GLUCOSA Sèrum',...","[{'imc': '0', 'anorexia': '0', 'perduaPes': '1...","[{'activitat': '3', 'estatMental': '1', 'humit...","[{'dataValoracio': '2020-07-30', 'horaValoraci...",[],"[{'codi': 'C09AA', 'dataInici': '2020-07-31 00...","[{'data': '20200731092102', 'disfagia': 'N', '...",...,None,6.9,13.9,None,7.86,40.6,0.32,38.7,None,78
8,90,F,0,"[{'date': '20220827', 'name': 'GLUCOSA Sèrum',...",[],"[{'activitat': '3', 'estatMental': '0', 'humit...",[],[],"[{'codi': 'N02BB', 'dataInici': '2023-07-17 00...",[],...,None,None,13.8,None,5.28,19.1,5.82,35.7,None,74
9,91,M,0,[],[],"[{'activitat': '1', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'C10AA', 'dataInici': '2020-08-25 00...",[],...,None,None,None,None,None,None,None,None,None,None


Sin aplicar la ultima condicion OR, es decir, queriendo nos devuelva solo las filas que cumplan las 3 condiciones, nos devuelve la: 4, 25, 42, 45 y 48. Sino, nos devuelve todas menos la 43, la cual no tiene un diagnostico de pneumonia.

Ahora que he distribuido los datos en 3 grupos, puedo empezar a analizar su contenido


## Número

In [1489]:
num_CON_PA = len(CON_PA)
num_CON_PA_MECVV= len(CON_PA_MECVV)
num_SIN_PA = len(SIN_PA)

print(num_CON_PA, num_CON_PA_MECVV, num_SIN_PA)

17 10 49


Esto también se puede ver con tan solo mirar las filas que han salido en el apartado anterior, donde vemos que la tabla de CON_PA tiene 17 filas, la de CON_PA_MECVV tiene 10 y la de SIN_PA tiene 49.

#### Pvalor

In [1490]:
z_stat, p_val = proportions_ztest(num_CON_PA, num_CON_PA_MECVV, value=0)
print("P-value para CON_PA vs CON_PA_MECVV:", p_val)
#TODO: es incorrecto?

P-value para CON_PA vs CON_PA_MECVV: nan


C:\Users\maria\PycharmProjects\practicum\.venv\Lib\site-packages\statsmodels\stats\proportion.py:1025: RuntimeWarning: invalid value encountered in sqrt
  std_diff = np.sqrt(var_)


In [1491]:
z_stat, p_val = proportions_ztest(num_CON_PA, num_SIN_PA, value=0)
print("P-value para CON_PA vs SIN_PA:", p_val)

P-value para CON_PA vs SIN_PA: 3.359355153724907e-07


In [1492]:
z_stat, p_val = proportions_ztest(num_CON_PA_MECVV, num_SIN_PA, value=0)
print("P-value para CON_PA_MECVV vs SIN_PA:", p_val)

P-value para CON_PA_MECVV vs SIN_PA: 0.0003932275362231457


## Edat

In [1493]:
edad_CON_PA = CON_PA['edat'].mean()
desviacion_estandar_edad = CON_PA['edat'].std()

resultado = f"{edad_CON_PA:.2f} ± {desviacion_estandar_edad:.2f}"
print("Promedio ± Desviación Estándar de la Edad:", resultado)


Promedio ± Desviación Estándar de la Edad: 87.06 ± 8.99


In [1494]:
edad_CON_PA_MECVV = CON_PA_MECVV['edat'].mean()
desviacion_estandar_edad = CON_PA_MECVV['edat'].std()

resultado = f"{edad_CON_PA_MECVV:.2f} ± {desviacion_estandar_edad:.2f}"
print("Promedio ± Desviación Estándar de la Edad:", resultado)

Promedio ± Desviación Estándar de la Edad: 88.90 ± 8.67


In [1495]:
edad_SIN_PA = SIN_PA['edat'].mean()
desviacion_estandar_edad = SIN_PA['edat'].std()

resultado = f"{edad_SIN_PA:.2f} ± {desviacion_estandar_edad:.2f}"
print("Promedio ± Desviación Estándar de la Edad:", resultado)

Promedio ± Desviación Estándar de la Edad: 80.12 ± 17.01


#### Pvalor

In [1496]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['edat'], CON_PA_MECVV['edat'], equal_var=False)
print("P-valor entre CON_PA y CON_PA_MECVV:", p_val_12)


P-valor entre CON_PA y CON_PA_MECVV: 0.6051223082851969


In [1497]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['edat'], SIN_PA['edat'], equal_var=False)
print("P-valor entre CON_PA y SIN_PA", p_val_13)

P-valor entre CON_PA y SIN_PA 0.038275970403095716


In [1498]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['edat'], SIN_PA['edat'], equal_var=False)
print("P-valor entre CON_PA_MECVV y SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV y SIN_PA: 0.024173860274705212


## Sexe

In [1499]:
sexe_CON_PA = CON_PA['sexe'].value_counts()
sexe_CON_PA_MECVV = CON_PA_MECVV['sexe'].value_counts()
sexe_SIN_PA = SIN_PA['sexe'].value_counts()

In [1500]:
if 'F' in sexe_CON_PA:
    percent_female1 = (sexe_CON_PA['F'] / len(CON_PA)) * 100
    print(f"Porcentaje de sexo femenino en CON_PA: {percent_female1:.2f}%")
else:
    print("No hay datos de sexo femenino en el CON_PA")

Porcentaje de sexo femenino en CON_PA: 47.06%


In [1501]:
if 'F' in sexe_CON_PA_MECVV:
    percent_female2 = (sexe_CON_PA_MECVV['F'] / len(CON_PA)) * 100
    print(f"Porcentaje de sexo femenino en DataFrame 2: {percent_female2:.2f}%")
else:
    print("No hay datos de sexo femenino en el DataFrame 2.")

Porcentaje de sexo femenino en DataFrame 2: 23.53%


In [1502]:
if 'F' in sexe_SIN_PA:
    percent_female3 = (sexe_SIN_PA['F'] / len(SIN_PA)) * 100
    print(f"Porcentaje de sexo femenino en DataFrame 3: {percent_female3:.2f}%")
else:
    print("No hay datos de sexo femenino en el DataFrame 3.")


Porcentaje de sexo femenino en DataFrame 3: 34.69%


#### Pvalor

In [1503]:
persexe_CON_PA = 47.06
persexe_CON_PA_MECVV = 23.53
persexe_SIN_PA= 34.69

In [1504]:
z_stat, p_val = proportions_ztest([47.06, 23.53], [100, 100])
print("P-value para CON_PA vs CON_PA_MECVV:", p_val)

P-value para CON_PA vs CON_PA_MECVV: 0.0004983897114131808


In [1505]:
z_stat, p_val = proportions_ztest([47.06, 34.69], [100, 100])
print("P-value para CON_PA vs SIN_PA:", p_val)

P-value para CON_PA vs SIN_PA: 0.0751965905655568


In [1506]:
z_stat, p_val = proportions_ztest([23.53, 34.69], [100, 100])
print("P-value para CON_PA_MECVV vs SIN_PA:", p_val)

P-value para CON_PA_MECVV vs SIN_PA: 0.08236190815486331


## Ingressos

In [1507]:
promedio_ingresos = CON_PA['Num_ingresos'].mean()
desviacion_estandar_ingresos = CON_PA['Num_ingresos'].std()

resultado = f"{promedio_ingresos:.2f} ± {desviacion_estandar_ingresos:.2f}"
print("Promedio ± Desviación Estándar del Número de Ingresos:", resultado)

Promedio ± Desviación Estándar del Número de Ingresos: 5.59 ± 3.37


In [1508]:
promedio_ingresos = CON_PA_MECVV['Num_ingresos'].mean()
desviacion_estandar_ingresos = CON_PA_MECVV['Num_ingresos'].std()

resultado = f"{promedio_ingresos:.2f} ± {desviacion_estandar_ingresos:.2f}"
print("Promedio ± Desviación Estándar del Número de Ingresos:", resultado)

Promedio ± Desviación Estándar del Número de Ingresos: 5.90 ± 2.08


In [1509]:
promedio_ingresos = SIN_PA['Num_ingresos'].mean()
desviacion_estandar_ingresos = SIN_PA['Num_ingresos'].std()

resultado = f"{promedio_ingresos:.2f} ± {desviacion_estandar_ingresos:.2f}"
print("Promedio ± Desviación Estándar del Número de Ingresos:", resultado)

Promedio ± Desviación Estándar del Número de Ingresos: 6.69 ± 5.13


#### Pvalor

In [1510]:
#ANOVA 
ingresos_CON_PA = CON_PA['Num_ingresos']
ingresos_CON_PA_MECVV = CON_PA_MECVV['Num_ingresos']
ingresos_SIN_PA = SIN_PA['Num_ingresos']

f_stat, p_valor = stats.f_oneway(ingresos_CON_PA, ingresos_CON_PA_MECVV)

print("Estadística F:", f_stat)
print("Valor p:", p_valor)

# Interpretación del resultado
alpha = 0.05
if p_valor < alpha:
    print("Hay diferencias significativas entre los grupos.")
else:
    print("No hay diferencias significativas entre los grupos.")

Estadística F: 0.06922327005897648
Valor p: 0.7946273649012201
No hay diferencias significativas entre los grupos.


In [1511]:
#Tukey
#TODO: es incorrecto?
CON_PA['grupo'] = 'CON_PA'
CON_PA_MECVV['grupo'] = 'CON_PA_MECVV'
SIN_PA['grupo'] = 'SIN_PA'

data = pd.concat([CON_PA, CON_PA_MECVV, SIN_PA], ignore_index=True)

# Realizar el test de Tukey solo si hay al menos dos grupos con datos
if len(data['grupo'].unique()) >= 2:
    # Realizar el test de Tukey
    tukey_results = pairwise_tukeyhsd(data['Num_ingresos'], data['grupo'], alpha=0.05)
    print(tukey_results)
else:
    print("Se requieren al menos dos grupos con datos para realizar comparaciones múltiples.")


C:\Users\maria\AppData\Local\Temp\ipykernel_10288\1477159889.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CON_PA['grupo'] = 'CON_PA'
C:\Users\maria\AppData\Local\Temp\ipykernel_10288\1477159889.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CON_PA_MECVV['grupo'] = 'CON_PA_MECVV'
C:\Users\maria\AppData\Local\Temp\ipykernel_10288\1477159889.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

      Multiple Comparison of Means - Tukey HSD, FWER=0.05      
   group1       group2    meandiff p-adj   lower  upper  reject
---------------------------------------------------------------
      CON_PA CON_PA_MECVV   0.3118 0.9836 -3.9892 4.6127  False
      CON_PA       SIN_PA   1.1056 0.6603 -1.9321 4.1434  False
CON_PA_MECVV       SIN_PA   0.7939 0.8681  -2.951 4.5387  False
---------------------------------------------------------------


In [1512]:
#T-test

In [1513]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['Num_ingresos'], CON_PA_MECVV['Num_ingresos'], equal_var=False)
print("P-valor entre CON_PA y CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA y CON_PA_MECVV: 0.7689185970171732


In [1514]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['Num_ingresos'], SIN_PA['Num_ingresos'], equal_var=False)
print("P-valor entre CON_PA y SIN_PA:", p_val_13)

P-valor entre CON_PA y SIN_PA: 0.3199162306439434


In [1515]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['Num_ingresos'], SIN_PA['Num_ingresos'], equal_var=False)
print("P-valor entre CON_PA_MECVV y SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV y SIN_PA: 0.4255873112788504


## Disfàgia (ICD de disfàgia)

In [1516]:
disf_CON_PA= (CON_PA['DO_diagnosticada'] == 1.0).mean() * 100
print(f"Porcentaje de disfagia: {disf_CON_PA:.2f}%")

Porcentaje de disfagia: 47.06%


In [1517]:
disf_CON_PA_MECVV = (CON_PA_MECVV['DO_diagnosticada'] == 1.0).mean() * 100
print(f"Porcentaje de disfagia: {disf_CON_PA_MECVV:.2f}%")

Porcentaje de disfagia: 30.00%


In [1518]:
disf_SIN_PA= (SIN_PA['DO_diagnosticada'] == 1.0).mean() * 100
print(f"Porcentaje de disfagia: {disf_SIN_PA:.2f}%")

Porcentaje de disfagia: 18.37%


#### Pvalor

In [1519]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['DO_diagnosticada'] == 1.0, CON_PA_MECVV['DO_diagnosticada'] == 1.0, equal_var=False)
print("P-valor entre CON_PA y CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA y CON_PA_MECVV: 0.3973619011700509


In [1520]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['DO_diagnosticada'] == 1.0, SIN_PA['Num_ingresos'] == 1.0, equal_var=False)
print("P-valor entre CON_PA y SIN_PA:", p_val_13)

P-valor entre CON_PA y SIN_PA: 0.0016711485988959095


In [1521]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['DO_diagnosticada'] == 1.0, SIN_PA['DO_diagnosticada'] == 1.0, equal_var=False)
print("P-valor entre CON_PA_MECVV y SIN_PA:", p_val_12)

P-valor entre CON_PA_MECVV y SIN_PA: 0.3973619011700509


In [1522]:
#Comprovacion con Tukey
CON_PA['grupo'] = 'CON_PA'
CON_PA_MECVV['grupo'] = 'CON_PA_MECVV'
SIN_PA['grupo'] = 'SIN_PA'

data = pd.concat([CON_PA, CON_PA_MECVV, SIN_PA], ignore_index=True)

# Realizar el test de Tukey solo si hay al menos dos grupos con datos
if len(data['grupo'].unique()) >= 2:
    # Realizar el test de Tukey
    tukey_results = pairwise_tukeyhsd(data['DO_diagnosticada'] == 1.0, data['grupo'], alpha=0.05)
    print(tukey_results)
else:
    print("Se requieren al menos dos grupos con datos para realizar comparaciones múltiples.")

C:\Users\maria\AppData\Local\Temp\ipykernel_10288\1554346727.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CON_PA['grupo'] = 'CON_PA'
C:\Users\maria\AppData\Local\Temp\ipykernel_10288\1554346727.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CON_PA_MECVV['grupo'] = 'CON_PA_MECVV'
C:\Users\maria\AppData\Local\Temp\ipykernel_10288\1554346727.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

      Multiple Comparison of Means - Tukey HSD, FWER=0.05      
   group1       group2    meandiff p-adj   lower  upper  reject
---------------------------------------------------------------
      CON_PA CON_PA_MECVV  -0.1706 0.5863 -0.5834 0.2422  False
      CON_PA       SIN_PA  -0.2869 0.0547 -0.5785 0.0046  False
CON_PA_MECVV       SIN_PA  -0.1163 0.7199 -0.4757 0.2431  False
---------------------------------------------------------------


## Index de Barthel

### Promedio

In [1523]:
promedio_barthel = CON_PA['Barthel_resultados'].mean()
desviacion_estandar_barthel = CON_PA['Barthel_resultados'].std()

resultado = f"{promedio_barthel:.2f} ± {desviacion_estandar_barthel:.2f}"
print("Promedio ± Desviación Estándar del Barthel:", resultado)

Promedio ± Desviación Estándar del Barthel: 22.65 ± 24.63


In [1524]:
promedio_barthel = CON_PA_MECVV['Barthel_resultados'].mean()
desviacion_estandar_barthel = CON_PA_MECVV['Barthel_resultados'].std()

resultado = f"{promedio_barthel:.2f} ± {desviacion_estandar_barthel:.2f}"
print("Promedio ± Desviación Estándar del Barthel:", resultado)

Promedio ± Desviación Estándar del Barthel: 28.50 ± 22.24


In [1525]:
promedio_barthel = SIN_PA['Barthel_resultados'].mean()
desviacion_estandar_barthel = SIN_PA['Barthel_resultados'].std()

resultado = f"{promedio_barthel:.2f} ± {desviacion_estandar_barthel:.2f}"
print("Promedio ± Desviación Estándar del Barthel:", resultado)

Promedio ± Desviación Estándar del Barthel: 48.88 ± 37.42


#### Pvalor

In [1526]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['Barthel_resultados'], CON_PA_MECVV['Barthel_resultados'], equal_var=False)
print("P-valor entre CON_PA y CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA y CON_PA_MECVV: 0.5328822564529123


In [1527]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['Barthel_resultados'], SIN_PA['Barthel_resultados'], equal_var=False)
print("P-valor entre CON_PA y SIN_PA:", p_val_13)

P-valor entre CON_PA y SIN_PA: 0.002116116594672647


In [1528]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['Barthel_resultados'], SIN_PA['Barthel_resultados'], equal_var=False)
print("P-valor entre CON_PA_MECVV y SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV y SIN_PA: 0.03132528496286713


### Valores de dependencia

##### CON_PA

In [1529]:
indepe_CON_PA = (CON_PA['Barthel_resultados'] == 100.0).mean() * 100
print(f"Porcentaje de indepe_CON_PA: {indepe_CON_PA:.2f}%")

Porcentaje de indepe_CON_PA: 0.00%


In [1530]:
mod_CON_PA = (CON_PA['Barthel_resultados'].between(61, 95)).mean() * 100
print(f"Porcentaje de mod_CON_PA: {mod_CON_PA:.2f}%")

Porcentaje de mod_CON_PA: 5.88%


In [1531]:
sev_CON_PA = (CON_PA['Barthel_resultados'].between(21, 60)).mean() * 100
print(f"Porcentaje de sev_CON_PA: {sev_CON_PA:.2f}%")

Porcentaje de sev_CON_PA: 41.18%


In [1532]:
total_CON_PA = (CON_PA['Barthel_resultados'] <= 20.0 ).mean() * 100
print(f"Porcentaje de total_CON_PA: {total_CON_PA:.2f}%")

Porcentaje de total_CON_PA: 52.94%


##### CON_PA_MECVV

In [1533]:
indepe_CON_PA_MECVV = (CON_PA_MECVV['Barthel_resultados'] == 100.0).mean() * 100
print(f"Porcentaje de indepe_CON_PA_MECVV: {indepe_CON_PA_MECVV:.2f}%")

Porcentaje de indepe_CON_PA_MECVV: 0.00%


In [1534]:
mod_CON_PA_MECVV = (CON_PA_MECVV['Barthel_resultados'].between(61,95)).mean() * 100
print(f"Porcentaje de mod_CON_PA_MECVV: {mod_CON_PA_MECVV:.2f}%")

Porcentaje de mod_CON_PA_MECVV: 0.00%


In [1535]:
sev_CON_PA_MECVV = (CON_PA_MECVV['Barthel_resultados'].between(21,60)).mean() * 100
print(f"Porcentaje de sev_CON_PA_MECVV: {sev_CON_PA_MECVV:.2f}%")

Porcentaje de sev_CON_PA_MECVV: 60.00%


In [1536]:
total_CON_PA_MECVV = (CON_PA_MECVV['Barthel_resultados'] <= 20.0).mean() * 100
print(f"Porcentaje de total_CON_PA_MECVV: {total_CON_PA_MECVV:.2f}%")

Porcentaje de total_CON_PA_MECVV: 40.00%


##### SIN_PA

In [1537]:
indepe_SIN_PA = (SIN_PA['Barthel_resultados'] == 100.0).mean() * 100
print(f"Porcentaje de indepe_SIN_PA: {indepe_SIN_PA:.2f}%")

Porcentaje de indepe_SIN_PA: 20.41%


In [1538]:
mod_SIN_PA = (SIN_PA['Barthel_resultados'].between(61,95)).mean() * 100
print(f"Porcentaje de mod_SIN_PA: {mod_SIN_PA:.2f}%")

Porcentaje de mod_SIN_PA: 18.37%


In [1539]:
sev_SIN_PA = (SIN_PA['Barthel_resultados'].between(21,60)).mean() * 100
print(f"Porcentaje de sev_SIN_PA: {sev_SIN_PA:.2f}%")

Porcentaje de sev_SIN_PA: 28.57%


In [1540]:
total_SIN_PA = (SIN_PA['Barthel_resultados'] <= 20).mean() * 100
print(f"Porcentaje de total_SIN_PA: {total_SIN_PA:.2f}%")

Porcentaje de total_SIN_PA: 32.65%


#### Pvalor

##### Independencia

In [1541]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['Barthel_resultados'] == 100.0, CON_PA_MECVV['Barthel_resultados'] == 100.0, equal_var=False)
print("P-valor entre indepe_CON_PA y indepe_CON_PA_MECVV:", p_val_12)

P-valor entre indepe_CON_PA y indepe_CON_PA_MECVV: nan


In [1542]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['Barthel_resultados'] == 100.0, SIN_PA['Barthel_resultados'] == 100.0, equal_var=False)
print("P-valor entre indepe_CON_PA y indepe_SIN_PA:", p_val_13)

P-valor entre indepe_CON_PA y indepe_SIN_PA: 0.0009905991330133408


In [1543]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['Barthel_resultados'] == 100.0, SIN_PA['Barthel_resultados'] == 100.0, equal_var=False)
print("P-valor entre indepe_CON_PA_MECVV y SIN_PA:", p_val_23)

P-valor entre indepe_CON_PA_MECVV y SIN_PA: 0.0009905991330133408


##### Moderada

In [1544]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['Barthel_resultados'].between(61, 95), CON_PA_MECVV['Barthel_resultados'].between(61, 95), equal_var=False)
print("P-valor entre mod_CON_PA y mod_CON_PA_MECVV:", p_val_12)

P-valor entre mod_CON_PA y mod_CON_PA_MECVV: 0.3321949846529795


In [1545]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['Barthel_resultados'].between(61, 95), SIN_PA['Barthel_resultados'].between(61, 95), equal_var=False)
print("P-valor entre indepe_CON_PA y mod_SIN_PA:", p_val_13)

P-valor entre indepe_CON_PA y mod_SIN_PA: 0.1308021277917621


In [1546]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['Barthel_resultados'].between(61, 95), SIN_PA['Barthel_resultados'].between(61, 95), equal_var=False)
print("P-valor entre mod_CON_PA_MECVV y mod_SIN_PA :", p_val_23)

P-valor entre mod_CON_PA_MECVV y mod_SIN_PA : 0.0019018439578946838


##### Severa

In [1547]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['Barthel_resultados'].between(21,60), CON_PA_MECVV['Barthel_resultados'].between(21,60), equal_var=False)
print("P-valor entre sev_CON_PA y sev_CON_PA_MECVV:", p_val_12)

P-valor entre sev_CON_PA y sev_CON_PA_MECVV: 0.36895172812187915


In [1548]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['Barthel_resultados'].between(21,60), SIN_PA['Barthel_resultados'].between(21,60), equal_var=False)
print("P-valor entre sev_CON_PA y sev_SIN_PA:", p_val_13)

P-valor entre sev_CON_PA y sev_SIN_PA: 0.37379734622047833


In [1549]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['Barthel_resultados'].between(21,60), SIN_PA['Barthel_resultados'].between(21,60), equal_var=False)
print("P-valor entre sev_CON_PA_MECVV y sev_SIN_PA:", p_val_23)

P-valor entre sev_CON_PA_MECVV y sev_SIN_PA: 0.09905259977964401


##### Total

In [1550]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['Barthel_resultados'] <= 20.0, CON_PA_MECVV['Barthel_resultados'] <= 20.0, equal_var=False)
print("P-valor entre total_CON_PA y total_CON_PA_MECVV:", p_val_12)

P-valor entre total_CON_PA y total_CON_PA_MECVV: 0.536420664203854


In [1551]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['Barthel_resultados'] <= 20.0, SIN_PA['Barthel_resultados'] <= 20.0, equal_var=False)
print("P-valor entre total_CON_PA y total_SIN_PA:", p_val_13)

P-valor entre total_CON_PA y total_SIN_PA: 0.16484379406840946


In [1552]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['Barthel_resultados'] <= 20.0, SIN_PA['Barthel_resultados'] <= 20.0, equal_var=False)
print("P-valor entre total_CON_PA_MECVV y total_SIN_PA:", p_val_23)

P-valor entre total_CON_PA_MECVV y total_SIN_PA: 0.6848541420862003


## Index del MNA

Primero hay que convertir la columna a tipo numerico ya que es de tipo "object" y no permite realizar el promedio

In [1553]:
CON_PA.loc[:, 'mna_resultats'] = pd.to_numeric(CON_PA['mna_resultats'], errors='coerce')
CON_PA_MECVV.loc[:, 'mna_resultats'] = pd.to_numeric(CON_PA_MECVV['mna_resultats'], errors='coerce')
SIN_PA.loc[:, 'mna_resultats'] = pd.to_numeric(SIN_PA['mna_resultats'], errors='coerce')
#TODO: es incorrecto?

### Promedio

In [1554]:
promedio_mna = CON_PA['mna_resultats'].mean()
desviacion_estandar_mna = CON_PA['mna_resultats'].std()

resultado = f"{promedio_mna:.2f} ± {desviacion_estandar_mna:.2f}"
print("Promedio ± Desviación Estándar del mna:", resultado)

Promedio ± Desviación Estándar del mna: 0.00 ± 0.00


In [1555]:
promedio_mna = CON_PA_MECVV['mna_resultats'].mean()
desviacion_estandar_mna = CON_PA_MECVV['mna_resultats'].std()

resultado = f"{promedio_mna:.2f} ± {desviacion_estandar_mna:.2f}"
print("Promedio ± Desviación Estándar del mna:", resultado)

Promedio ± Desviación Estándar del mna: 0.00 ± nan


In [1556]:
promedio_mna = SIN_PA['mna_resultats'].mean()
desviacion_estandar_mna = SIN_PA['mna_resultats'].std()

resultado = f"{promedio_mna:.2f} ± {desviacion_estandar_mna:.2f}"
print("Promedio ± Desviación Estándar del mna:", resultado)

Promedio ± Desviación Estándar del mna: 3.05 ± 5.92


#### Pvalor

In [1557]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['mna_resultats'], CON_PA_MECVV['mna_resultats'], equal_var=False)
print("P-valor entre CON_PA y CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA y CON_PA_MECVV: nan


In [1558]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['mna_resultats'], SIN_PA['mna_resultats'], equal_var=False)
print("P-valor entre CON_PA y SIN_PA:", p_val_13)

P-valor entre CON_PA y SIN_PA: nan


In [1559]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['mna_resultats'], SIN_PA['mna_resultats'], equal_var=False)
print("P-valor entre CON_PA_MECVV y SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV y SIN_PA: nan


### Valores de malnutricion

##### CON_PA

In [1560]:
norm_CON_PA = CON_PA['mna_resultats'].between(24, 30)
porcentaje_norm_CON_PA = norm_CON_PA.mean() * 100
num_observaciones_norm_CON_PA = norm_CON_PA.sum()

print(f"Porcentaje de norm_CON_PA: {porcentaje_norm_CON_PA:.2f}%")
print(f"Número de observaciones en norm_CON_PA: {num_observaciones_norm_CON_PA}")

Porcentaje de norm_CON_PA: 0.00%
Número de observaciones en norm_CON_PA: 0


In [1561]:
malnutr_CON_PA = CON_PA['mna_resultats'].between(17, 23.5)
porcentaje_malnutr_CON_PA = malnutr_CON_PA.mean() * 100
num_observaciones_malnutr_CON_PA = norm_CON_PA.sum()

print(f"Porcentaje de malnutr_CON_PA: {porcentaje_malnutr_CON_PA:.2f}%")
print(f"Número de observaciones en norm_CON_PA: {num_observaciones_malnutr_CON_PA}")

Porcentaje de malnutr_CON_PA: 0.00%
Número de observaciones en norm_CON_PA: 0


In [1562]:
malnut_CON_PA = CON_PA['mna_resultats'] < 17
porcentaje_malnut_CON_PA = malnut_CON_PA.mean() * 100
num_observaciones_malnut_CON_PA = malnut_CON_PA.sum()

print(f"Porcentaje de malnut_CON_PA: {porcentaje_malnut_CON_PA:.2f}%")
print(f"Número de observaciones en malnut_CON_PA: {num_observaciones_malnut_CON_PA}")

Porcentaje de malnut_CON_PA: 23.53%
Número de observaciones en malnut_CON_PA: 4


##### CON_PA_MECVV

In [1563]:
norm_CON_PA_MECVV = CON_PA_MECVV['mna_resultats'].between(24, 30)
porcentaje_norm_CON_PA_MECVV = norm_CON_PA_MECVV.mean() * 100
num_observaciones_norm_CON_PA_MECVV = norm_CON_PA_MECVV.sum()

print(f"Porcentaje de norm_CON_PA_MECVV: {porcentaje_norm_CON_PA_MECVV:.2f}%")
print(f"Número de observaciones en norm_CON_PA_MECVV: {num_observaciones_norm_CON_PA_MECVV}")

Porcentaje de norm_CON_PA_MECVV: 0.00%
Número de observaciones en norm_CON_PA_MECVV: 0


In [1564]:
malnutr_CON_PA_MECVV = CON_PA_MECVV['mna_resultats'].between(17, 23.5)
porcentaje_malnutr_CON_PA_MECVV = malnutr_CON_PA_MECVV.mean() * 100
num_observaciones_malnutr_CON_PA_MECVV = norm_CON_PA_MECVV.sum()

print(f"Porcentaje de malnutr_CON_PA_MECVV: {porcentaje_malnutr_CON_PA_MECVV:.2f}%")
print(f"Número de observaciones en norm_CON_PA_MECVV: {num_observaciones_malnutr_CON_PA_MECVV}")

Porcentaje de malnutr_CON_PA_MECVV: 0.00%
Número de observaciones en norm_CON_PA_MECVV: 0


In [1565]:
malnut_CON_PA_MECVV = CON_PA_MECVV['mna_resultats'] < 17
porcentaje_malnut_CON_PA_MECVV = malnut_CON_PA_MECVV.mean() * 100
num_observaciones_malnut_CON_PA_MECVV = malnut_CON_PA_MECVV.sum()

print(f"Porcentaje de malnut_CON_PA_MECVV: {porcentaje_malnut_CON_PA_MECVV:.2f}%")
print(f"Número de observaciones en malnut_CON_PA_MECVV: {num_observaciones_malnut_CON_PA_MECVV}")

Porcentaje de malnut_CON_PA_MECVV: 10.00%
Número de observaciones en malnut_CON_PA_MECVV: 1


##### SIN_PA

In [1566]:
norm_SIN_PA = SIN_PA['mna_resultats'].between(24, 30)
porcentaje_norm_SIN_PA = norm_SIN_PA.mean() * 100
num_observaciones_norm_SIN_PA = norm_SIN_PA.sum()

print(f"Porcentaje de norm_SIN_PA: {porcentaje_norm_SIN_PA:.2f}%")
print(f"Número de observaciones en norm_SIN_PA: {num_observaciones_norm_SIN_PA}")

Porcentaje de norm_SIN_PA: 0.00%
Número de observaciones en norm_SIN_PA: 0


In [1567]:
malnutr_SIN_PA = SIN_PA['mna_resultats'].between(17, 23.5)
porcentaje_malnutr_SIN_PA = malnutr_SIN_PA.mean() * 100
num_observaciones_malnutr_SIN_PA = malnutr_SIN_PA.sum()

print(f"Porcentaje de malnutr_SIN_PA: {porcentaje_malnutr_SIN_PA:.2f}%")
print(f"Número de observaciones en malnutr_SIN_PA: {num_observaciones_malnutr_SIN_PA}")

Porcentaje de malnutr_SIN_PA: 2.04%
Número de observaciones en malnutr_SIN_PA: 1


In [1568]:
malnut_SIN_PA = SIN_PA['mna_resultats'] < 17
porcentaje_malnut_SIN_PA = malnut_SIN_PA.mean() * 100
num_observaciones_malnut_SIN_PA = malnut_SIN_PA.sum()

print(f"Porcentaje de malnut_SIN_PA: {porcentaje_malnut_SIN_PA:.2f}%")
print(f"Número de observaciones en malnut_SIN_PA: {num_observaciones_malnut_SIN_PA}")

Porcentaje de malnut_SIN_PA: 18.37%
Número de observaciones en malnut_SIN_PA: 9


#### Pvalor

##### Normal

In [1569]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['mna_resultats'].between(24, 30), CON_PA_MECVV['mna_resultats'].between(24, 30), equal_var=False)
print("P-valor entre norm_CON_PA y norm_CON_PA_MECVV:", p_val_12)

P-valor entre norm_CON_PA y norm_CON_PA_MECVV: nan


In [1570]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['mna_resultats'].between(24, 30), SIN_PA['mna_resultats'].between(24, 30), equal_var=False)
print("P-valor entre norm_CON_PA y norm_SIN_PA:", p_val_13)

P-valor entre norm_CON_PA y norm_SIN_PA: nan


In [1571]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['mna_resultats'].between(24, 30), SIN_PA['mna_resultats'].between(24, 30), equal_var=False)
print("P-valor entre norm_CON_PA_MECVV y norm_SIN_PA:", p_val_23)

P-valor entre norm_CON_PA_MECVV y norm_SIN_PA: nan


##### Riesgo de malnutricion

In [1572]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['mna_resultats'].between(17, 23.5), CON_PA_MECVV['mna_resultats'].between(17, 23.5), equal_var=False)
print("P-valor entre malnutr_CON_PA y malnutr_CON_PA_MECVV:", p_val_12)

P-valor entre malnutr_CON_PA y malnutr_CON_PA_MECVV: nan


In [1573]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['mna_resultats'].between(17, 23.5), SIN_PA['mna_resultats'].between(17, 23.5), equal_var=False)
print("P-valor entre malnutr_CON_PA y malnutr_SIN_PA:", p_val_13)

P-valor entre malnutr_CON_PA y malnutr_SIN_PA: 0.3223252185211165


In [1574]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['mna_resultats'].between(17, 23.5), SIN_PA['mna_resultats'].between(17, 23.5), equal_var=False)
print("P-valor entre malnutr_CON_PA_MECVV y malnutr_SIN_PA:", p_val_23)

P-valor entre malnutr_CON_PA_MECVV y malnutr_SIN_PA: 0.3223252185211165


##### Malnutricion

In [1575]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['mna_resultats'] <17 , 
                                CON_PA_MECVV['mna_resultats'] < 17, equal_var=False)
print("P-valor entre malnut_CON_PA y norm_CON_PA_MECVV:", p_val_12)

P-valor entre malnut_CON_PA y norm_CON_PA_MECVV: 0.3626443145321582


In [1576]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['mna_resultats'] <17 , 
                                SIN_PA['mna_resultats'] < 17, equal_var=False)
print("P-valor entre malnut_CON_PA y malnut_SIN_PA:", p_val_13)

P-valor entre malnut_CON_PA y malnut_SIN_PA: 0.6703598116552147


In [1577]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['mna_resultats'] <17 , 
                                SIN_PA['mna_resultats'] < 17, equal_var=False)
print("P-valor entre malnut_CON_PA_MECVV y malnut_SIN_PA:", p_val_23)

P-valor entre malnut_CON_PA_MECVV y malnut_SIN_PA: 0.47623865054662595


## Indice de la  EMINA

### Promedio

In [1578]:
promedio_emina = CON_PA['EMINA_sumatorios_comparados'].mean()
desviacion_estandar_emina = CON_PA['EMINA_sumatorios_comparados'].std()

resultado = f"{promedio_emina:.2f} ± {desviacion_estandar_emina:.2f}"
print("Promedio ± Desviación Estándar de la emina:", resultado)

Promedio ± Desviación Estándar de la emina: 7.94 ± 3.21


In [1579]:
promedio_emina = CON_PA_MECVV['EMINA_sumatorios_comparados'].mean()
desviacion_estandar_emina = CON_PA_MECVV['EMINA_sumatorios_comparados'].std()

resultado = f"{promedio_emina:.2f} ± {desviacion_estandar_emina:.2f}"
print("Promedio ± Desviación Estándar de la emina:", resultado)

Promedio ± Desviación Estándar de la emina: 7.60 ± 2.01


In [1580]:
promedio_emina = SIN_PA['EMINA_sumatorios_comparados'].mean()
desviacion_estandar_emina = SIN_PA['EMINA_sumatorios_comparados'].std()

resultado = f"{promedio_emina:.2f} ± {desviacion_estandar_emina:.2f}"
print("Promedio ± Desviación Estándar de la emina:", resultado)

Promedio ± Desviación Estándar de la emina: 5.64 ± 4.08


#### Pvalor

In [1581]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['EMINA_sumatorios_comparados'], CON_PA_MECVV['EMINA_sumatorios_comparados'], equal_var=False)
print("P-valor entre CON_PA y CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA y CON_PA_MECVV: 0.7372065144531302


In [1582]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['EMINA_sumatorios_comparados'], SIN_PA['EMINA_sumatorios_comparados'], equal_var=False)
print("P-valor entre CON_PA y SIN_PA:", p_val_13)

P-valor entre CON_PA y SIN_PA: nan


In [1583]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['EMINA_sumatorios_comparados'], SIN_PA['EMINA_sumatorios_comparados'], equal_var=False)
print("P-valor entre CON_PA_MECVV y SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV y SIN_PA: nan


### Valores EMINA

##### CON_PA

In [1584]:
lowr_CON_PA = CON_PA['EMINA_sumatorios_comparados'].between(0, 5)
porcentaje_lowr_CON_PA = lowr_CON_PA.mean() * 100
num_observaciones_lowr_CON_PA = lowr_CON_PA.sum()

print(f"Porcentaje de lowr_CON_PA: {porcentaje_lowr_CON_PA:.2f}%")
print(f"Número de observaciones en lowr_CON_PA: {num_observaciones_lowr_CON_PA}")

Porcentaje de lowr_CON_PA: 17.65%
Número de observaciones en lowr_CON_PA: 3


In [1585]:
modr_CON_PA = CON_PA['EMINA_sumatorios_comparados'].between(6, 10)
porcentaje_modr_CON_PA = modr_CON_PA.mean() * 100
num_observaciones_modr_CON_PA = modr_CON_PA.sum()

print(f"Porcentaje de modr_CON_PA: {porcentaje_modr_CON_PA:.2f}%")
print(f"Número de observaciones en modr_CON_PA: {num_observaciones_modr_CON_PA}")

Porcentaje de modr_CON_PA: 64.71%
Número de observaciones en modr_CON_PA: 11


In [1586]:
highr_CON_PA = CON_PA['EMINA_sumatorios_comparados'].between(11, 15)
porcentaje_highr_CON_PA = highr_CON_PA.mean() * 100
num_observaciones_highr_CON_PA = highr_CON_PA.sum()

print(f"Porcentaje de highr_CON_PA: {porcentaje_highr_CON_PA:.2f}%")
print(f"Número de observaciones en highr_CON_PA: {num_observaciones_highr_CON_PA}")

Porcentaje de highr_CON_PA: 17.65%
Número de observaciones en highr_CON_PA: 3


##### CON_PA_MECVV

In [1587]:
lowr_CON_PA_MECVV = CON_PA_MECVV['EMINA_sumatorios_comparados'].between(0, 5)
porcentaje_lowr_CON_PA_MECVV = lowr_CON_PA_MECVV.mean() * 100
num_observaciones_lowr_CON_PA_MECVV = lowr_CON_PA_MECVV.sum()

print(f"Porcentaje de lowr_CON_PA_MECVV: {porcentaje_lowr_CON_PA_MECVV:.2f}%")
print(f"Número de observaciones en lowr_CON_PA_MECVV: {num_observaciones_lowr_CON_PA_MECVV}")

Porcentaje de lowr_CON_PA_MECVV: 20.00%
Número de observaciones en lowr_CON_PA_MECVV: 2


In [1588]:
modr_CON_PA_MECVV = CON_PA_MECVV['EMINA_sumatorios_comparados'].between(6, 10)
porcentaje_modr_CON_PA_MECVV = modr_CON_PA_MECVV.mean() * 100
num_observaciones_modr_CON_PA_MECVV = modr_CON_PA_MECVV.sum()

print(f"Porcentaje de modr_CON_PA_MECVV: {porcentaje_modr_CON_PA_MECVV:.2f}%")
print(f"Número de observaciones en modr_CON_PA_MECVV: {num_observaciones_modr_CON_PA_MECVV}")

Porcentaje de modr_CON_PA_MECVV: 80.00%
Número de observaciones en modr_CON_PA_MECVV: 8


In [1589]:
highr_CON_PA_MECVV = CON_PA_MECVV['EMINA_sumatorios_comparados'].between(11, 15)
porcentaje_highr_CON_PA_MECVV = highr_CON_PA_MECVV.mean() * 100
num_observaciones_highr_CON_PA_MECVV = highr_CON_PA_MECVV.sum()

print(f"Porcentaje de highr_CON_PA_MECVV: {porcentaje_highr_CON_PA_MECVV:.2f}%")
print(f"Número de observaciones en highr_CON_PA_MECVV: {num_observaciones_highr_CON_PA_MECVV}")

Porcentaje de highr_CON_PA_MECVV: 0.00%
Número de observaciones en highr_CON_PA_MECVV: 0


##### SIN_PA

In [1590]:
lowr_SIN_PA = SIN_PA['EMINA_sumatorios_comparados'].between(0, 5)
porcentaje_lowr_SIN_PA = lowr_SIN_PA.mean() * 100
num_observaciones_lowr_SIN_PA = lowr_SIN_PA.sum()

print(f"Porcentaje de lowr_SIN_PA: {porcentaje_lowr_SIN_PA:.2f}%")
print(f"Número de observaciones en lowr_SIN_PA: {num_observaciones_lowr_SIN_PA}")

Porcentaje de lowr_SIN_PA: 44.90%
Número de observaciones en lowr_SIN_PA: 22


In [1591]:
modr_SIN_PA = SIN_PA['EMINA_sumatorios_comparados'].between(6, 10)
porcentaje_modr_SIN_PA = modr_SIN_PA.mean() * 100
num_observaciones_modr_SIN_PA = modr_SIN_PA.sum()

print(f"Porcentaje de modr_SIN_PA: {porcentaje_modr_SIN_PA:.2f}%")
print(f"Número de observaciones en modr_SIN_PA: {num_observaciones_modr_SIN_PA}")

Porcentaje de modr_SIN_PA: 40.82%
Número de observaciones en modr_SIN_PA: 20


In [1592]:
highr_SIN_PA = SIN_PA['EMINA_sumatorios_comparados'].between(11, 15)
porcentaje_highr_SIN_PA = highr_SIN_PA.mean() * 100
num_observaciones_highr_SIN_PA = highr_SIN_PA.sum()

print(f"Porcentaje de highr_SIN_PA: {porcentaje_highr_SIN_PA:.2f}%")
print(f"Número de observaciones en highr_SIN_PA: {num_observaciones_highr_SIN_PA}")

Porcentaje de highr_SIN_PA: 10.20%
Número de observaciones en highr_SIN_PA: 5


#### Pvalor 

##### Riesgo bajo

In [1593]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['EMINA_sumatorios_comparados'].between(0,5), CON_PA_MECVV['EMINA_sumatorios_comparados'].between(0,5), equal_var=False)
print("P-valor entre CON_PA y CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA y CON_PA_MECVV: 0.8874457485314164


In [1594]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['EMINA_sumatorios_comparados'].between(0,5), SIN_PA['EMINA_sumatorios_comparados'].between(0,5), equal_var=False)
print("P-valor entre CON_PA y SIN_PA:", p_val_13)

P-valor entre CON_PA y SIN_PA: 0.028470032037651503


In [1595]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['EMINA_sumatorios_comparados'].between(0,5), SIN_PA['EMINA_sumatorios_comparados'].between(0,5), equal_var=False)
print("P-valor entre CON_PA_MECVV y SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV y SIN_PA: 0.12129243251471264


##### Riesgo moderado 

In [1596]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['EMINA_sumatorios_comparados'].between(6,10), CON_PA_MECVV['EMINA_sumatorios_comparados'].between(6,10), equal_var=False)
print("P-valor entre CON_PA y CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA y CON_PA_MECVV: 0.40237952012988454


In [1597]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['EMINA_sumatorios_comparados'].between(6,10), SIN_PA['EMINA_sumatorios_comparados'].between(6,10), equal_var=False)
print("P-valor entre CON_PA y SIN_PA:", p_val_13)

P-valor entre CON_PA y SIN_PA: 0.09654829012396034


In [1598]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['EMINA_sumatorios_comparados'].between(6,10), SIN_PA['EMINA_sumatorios_comparados'].between(6,10), equal_var=False)
print("P-valor entre CON_PA_MECVV y SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV y SIN_PA: 0.02066325652755892


##### Riesgo alto

In [1599]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['EMINA_sumatorios_comparados'].between(11,15), CON_PA_MECVV['EMINA_sumatorios_comparados'].between(11,15), equal_var=False)
print("P-valor entre CON_PA y CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA y CON_PA_MECVV: 0.08262239198015026


In [1600]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['EMINA_sumatorios_comparados'].between(11,15), SIN_PA['EMINA_sumatorios_comparados'].between(11,15), equal_var=False)
print("P-valor entre CON_PA y SIN_PA:", p_val_13)

P-valor entre CON_PA y SIN_PA: 0.48486212369214565


In [1601]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['EMINA_sumatorios_comparados'].between(11,15), SIN_PA['EMINA_sumatorios_comparados'].between(11,15), equal_var=False)
print("P-valor entre CON_PA_MECVV y SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV y SIN_PA: 0.02374418239053046


## Indice de la canadenca 

### Promedio

In [1602]:
promedio_canad = CON_PA['Canadenca_sumatorios_comparados'].mean()
desviacion_estandar_canad = CON_PA['Canadenca_sumatorios_comparados'].std()

resultado = f"{promedio_canad:.2f} ± {desviacion_estandar_canad:.2f}"
print("Promedio ± Desviación Estándar de la canadenca:", resultado)

Promedio ± Desviación Estándar de la canadenca: 7.00 ± nan


In [1603]:
promedio_canad = CON_PA_MECVV['Canadenca_sumatorios_comparados'].mean()
desviacion_estandar_canad = CON_PA_MECVV['Canadenca_sumatorios_comparados'].std()

resultado = f"{promedio_canad:.2f} ± {desviacion_estandar_canad:.2f}"
print("Promedio ± Desviación Estándar de la canadenca:", resultado)

Promedio ± Desviación Estándar de la canadenca: 7.00 ± nan


In [1604]:
promedio_canad = SIN_PA['Canadenca_sumatorios_comparados'].mean()
desviacion_estandar_canad = SIN_PA['Canadenca_sumatorios_comparados'].std()

resultado = f"{promedio_canad:.2f} ± {desviacion_estandar_canad:.2f}"
print("Promedio ± Desviación Estándar de la canadenca:", resultado)

Promedio ± Desviación Estándar de la canadenca: 8.00 ± 1.41


#### Pvalor

In [1605]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['Canadenca_sumatorios_comparados'], CON_PA_MECVV['Canadenca_sumatorios_comparados'], equal_var=False)
print("P-valor entre CON_PA y CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA y CON_PA_MECVV: nan


In [1606]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['Canadenca_sumatorios_comparados'], SIN_PA['Canadenca_sumatorios_comparados'], equal_var=False)
print("P-valor entre CON_PA y SIN_PA:", p_val_13)

P-valor entre CON_PA y SIN_PA: nan


In [1607]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['Canadenca_sumatorios_comparados'], SIN_PA['Canadenca_sumatorios_comparados'], equal_var=False)
print("P-valor entre CON_PA_MECVV y SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV y SIN_PA: nan


### Valores canadenca

##### CON_PA 

In [1608]:
mild_CON_PA = CON_PA['Canadenca_sumatorios_comparados'].between(10.5, 11.5)
porcentaje_mild_CON_PA = mild_CON_PA.mean() * 100
num_observaciones_mild_CON_PA = mild_CON_PA.sum()

print(f"Porcentaje de mild_CON_PA: {porcentaje_mild_CON_PA:.2f}%")
print(f"Número de observaciones en mild_CON_PA: {num_observaciones_mild_CON_PA}")

Porcentaje de mild_CON_PA: 0.00%
Número de observaciones en mild_CON_PA: 0


In [1609]:
modneu_CON_PA = CON_PA['Canadenca_sumatorios_comparados'].between(5,10)
porcentaje_modneu_CON_PA = modneu_CON_PA.mean() * 100
num_observaciones_modneu_CON_PA = modneu_CON_PA.sum()

print(f"Porcentaje de modneu_CON_PA: {porcentaje_modneu_CON_PA:.2f}%")
print(f"Número de observaciones en modneu_CON_PA: {num_observaciones_modneu_CON_PA}")

Porcentaje de modneu_CON_PA: 5.88%
Número de observaciones en modneu_CON_PA: 1


In [1610]:
sevneu_CON_PA = CON_PA['Canadenca_sumatorios_comparados'].between(1.5, 4.5)
porcentaje_sevneu_CON_PA = sevneu_CON_PA.mean() * 100
num_observaciones_sevneu_CON_PA = sevneu_CON_PA.sum()

print(f"Porcentaje de sevneu_CON_PA: {porcentaje_sevneu_CON_PA:.2f}%")
print(f"Número de observaciones en sevneu_CON_PA: {num_observaciones_sevneu_CON_PA}")

Porcentaje de sevneu_CON_PA: 0.00%
Número de observaciones en sevneu_CON_PA: 0


##### CON_PA_MECVV

In [1611]:
mild_CON_PA_MECVV = CON_PA_MECVV['Canadenca_sumatorios_comparados'].between(10.5, 11.5)
porcentaje_mild_CON_PA_MECVV = mild_CON_PA_MECVV.mean() * 100
num_observaciones_mild_CON_PA_MECVV = mild_CON_PA_MECVV.sum()

print(f"Porcentaje de mild_CON_PA_MECVV: {porcentaje_mild_CON_PA_MECVV:.2f}%")
print(f"Número de observaciones en mild_CON_PA_MECVV: {num_observaciones_mild_CON_PA_MECVV}")

Porcentaje de mild_CON_PA_MECVV: 0.00%
Número de observaciones en mild_CON_PA_MECVV: 0


In [1612]:
modneu_CON_PA_MECVV = CON_PA_MECVV['Canadenca_sumatorios_comparados'].between(5,10)
porcentaje_modneu_CON_PA_MECVV = modneu_CON_PA_MECVV.mean() * 100
num_observaciones_modneu_CON_PA_MECVV = modneu_CON_PA_MECVV.sum()

print(f"Porcentaje de modneu_CON_PA_MECVV: {porcentaje_modneu_CON_PA_MECVV:.2f}%")
print(f"Número de observaciones en modneu_CON_PA_MECVV: {num_observaciones_modneu_CON_PA_MECVV}")

Porcentaje de modneu_CON_PA_MECVV: 10.00%
Número de observaciones en modneu_CON_PA_MECVV: 1


In [1613]:
sevneu_CON_PA_MECVV = CON_PA_MECVV['Canadenca_sumatorios_comparados'].between(1.5, 4.5)
porcentaje_sevneu_CON_PA_MECVV = sevneu_CON_PA_MECVV.mean() * 100
num_observaciones_sevneu_CON_PA_MECVV = sevneu_CON_PA_MECVV.sum()

print(f"Porcentaje de sevneu_CON_PA_MECVV: {porcentaje_sevneu_CON_PA_MECVV:.2f}%")
print(f"Número de observaciones en sevneu_CON_PA_MECVV: {num_observaciones_sevneu_CON_PA_MECVV}")

Porcentaje de sevneu_CON_PA_MECVV: 0.00%
Número de observaciones en sevneu_CON_PA_MECVV: 0


##### SIN_PA

In [1614]:
mild_SIN_PA = SIN_PA['Canadenca_sumatorios_comparados'].between(10.5, 11.5)
porcentaje_mild_SIN_PA = mild_SIN_PA.mean() * 100
num_observaciones_mild_SIN_PA = mild_SIN_PA.sum()

print(f"Porcentaje de mild_SIN_PA: {porcentaje_mild_SIN_PA:.2f}%")
print(f"Número de observaciones en mild_SIN_PA: {num_observaciones_mild_SIN_PA}")

Porcentaje de mild_SIN_PA: 0.00%
Número de observaciones en mild_SIN_PA: 0


In [1615]:
modneu_SIN_PA = SIN_PA['Canadenca_sumatorios_comparados'].between(5,10)
porcentaje_modneu_SIN_PA = modneu_SIN_PA.mean() * 100
num_observaciones_modneu_SIN_PA = modneu_SIN_PA.sum()

print(f"Porcentaje de modneu_SIN_PA: {porcentaje_modneu_SIN_PA:.2f}%")
print(f"Número de observaciones en modneu_SIN_PA: {num_observaciones_modneu_SIN_PA}")

Porcentaje de modneu_SIN_PA: 4.08%
Número de observaciones en modneu_SIN_PA: 2


In [1616]:
sevneu_SIN_PA = SIN_PA['Canadenca_sumatorios_comparados'].between(1.5, 4.5)
porcentaje_sevneu_SIN_PA = sevneu_SIN_PA.mean() * 100
num_observaciones_sevneu_SIN_PA = sevneu_SIN_PA.sum()

print(f"Porcentaje de sevneu_SIN_PA: {porcentaje_sevneu_SIN_PA:.2f}%")
print(f"Número de observaciones en sevneu_SIN_PA: {num_observaciones_sevneu_SIN_PA}")

Porcentaje de sevneu_SIN_PA: 0.00%
Número de observaciones en sevneu_SIN_PA: 0


#### Pvalor

##### Deficit neurologico ligero (mild)

In [1617]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['Canadenca_sumatorios_comparados'].between(10.5,11.5), CON_PA_MECVV['Canadenca_sumatorios_comparados'].between(10.5,11.5), equal_var=False)
print("P-valor entre CON_PA y CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA y CON_PA_MECVV: nan


In [1618]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['Canadenca_sumatorios_comparados'].between(10.5,11.5), SIN_PA['Canadenca_sumatorios_comparados'].between(10.5,11.5), equal_var=False)
print("P-valor entre CON_PA y SIN_PA:", p_val_13)

P-valor entre CON_PA y SIN_PA: nan


In [1619]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['Canadenca_sumatorios_comparados'].between(10.5,11.5), SIN_PA['Canadenca_sumatorios_comparados'].between(10.5,11.5), equal_var=False)
print("P-valor entre CON_PA_MECVV y SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV y SIN_PA: nan


##### Deficit neurologico moderado

In [1620]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['Canadenca_sumatorios_comparados'].between(5,10), CON_PA_MECVV['Canadenca_sumatorios_comparados'].between(5, 10), equal_var=False)
print("P-valor entre CON_PA y CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA y CON_PA_MECVV: 0.727506051578171


In [1621]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['Canadenca_sumatorios_comparados'].between(5,10), SIN_PA['Canadenca_sumatorios_comparados'].between(5, 10), equal_var=False)
print("P-valor entre CON_PA y CON_PA_MECVV:", p_val_13)

P-valor entre CON_PA y CON_PA_MECVV: 0.7853810396215979


In [1622]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['Canadenca_sumatorios_comparados'].between(5,10), SIN_PA['Canadenca_sumatorios_comparados'].between(5, 10), equal_var=False)
print("P-valor entre CON_PA_MECVV y SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV y SIN_PA: 0.5812513721413167


##### Deficit neurologico severo

In [1623]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['Canadenca_sumatorios_comparados'].between(1.5,4.5), CON_PA_MECVV['Canadenca_sumatorios_comparados'].between(1.5,4.5), equal_var=False)
print("P-valor entre CON_PA y CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA y CON_PA_MECVV: nan


In [1624]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['Canadenca_sumatorios_comparados'].between(1.5, 4.5), SIN_PA['Canadenca_sumatorios_comparados'].between(1.5, 4.5), equal_var=False)
print("P-valor entre CON_PA y SIN_PA:", p_val_13)

P-valor entre CON_PA y SIN_PA: nan


In [1625]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['Canadenca_sumatorios_comparados'].between(1.5,4.5), SIN_PA['Canadenca_sumatorios_comparados'].between(1.5, 4.5), equal_var=False)
print("P-valor entre CON_PA_MECVV y SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV y SIN_PA: nan


## Infecciones respiratorias

##### CON_PA 

In [1626]:
mainresp_CON_PA= (CON_PA['Main_respiratory_infections_diagnosticada'] == 1.0).mean() * 100
print(f"Porcentaje de infecciones respiratorias principales : {mainresp_CON_PA:.2f}%")

Porcentaje de infecciones respiratorias principales : 41.18%


In [1627]:
LRTI_CON_PA= (CON_PA['LRTI_diagnosticada'] == 1.0).mean() * 100
print(f"Porcentaje de LRTI: {LRTI_CON_PA:.2f}%")

Porcentaje de LRTI: 17.65%


In [1628]:
COPD_CON_PA= (CON_PA['COPD_exacerbations_diagnosticada'] == 1.0).mean() * 100
print(f"Porcentaje de exacerbaciones de COPD: {COPD_CON_PA:.2f}%")

Porcentaje de exacerbaciones de COPD: 29.41%


In [1629]:
fibro_CON_PA= (CON_PA['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0).mean() * 100
print(f"Porcentaje de fibrosis pulmonar y fibrotorax: {fibro_CON_PA:.2f}%")

Porcentaje de fibrosis pulmonar y fibrotorax: 0.00%


##### CON_PA_MECVV

In [1630]:
mainresp_CON_PA_MECVV= (CON_PA_MECVV['Main_respiratory_infections_diagnosticada'] == 1.0).mean() * 100
print(f"Porcentaje de infecciones respiratorias principales : {mainresp_CON_PA_MECVV:.2f}%")

Porcentaje de infecciones respiratorias principales : 50.00%


In [1631]:
LRTI_CON_PA_MECVV= (CON_PA_MECVV['LRTI_diagnosticada'] == 1.0).mean() * 100
print(f"Porcentaje de LRTI: {LRTI_CON_PA_MECVV:.2f}%")

Porcentaje de LRTI: 20.00%


In [1632]:
COPD_CON_PA_MECVV = (CON_PA_MECVV['COPD_exacerbations_diagnosticada'] == 1.0).mean() * 100
print(f"Porcentaje de exacerbaciones de COPD: {COPD_CON_PA_MECVV:.2f}%")

Porcentaje de exacerbaciones de COPD: 30.00%


In [1633]:
fibro_CON_PA_MECVV = (CON_PA_MECVV['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0).mean() * 100
print(f"Porcentaje de fibrosis pulmonar y fibrotorax: {fibro_CON_PA_MECVV:.2f}%")

Porcentaje de fibrosis pulmonar y fibrotorax: 0.00%


##### SIN_PA

In [1634]:
mainresp_SIN_PA= (SIN_PA['Main_respiratory_infections_diagnosticada'] == 1.0).mean() * 100
print(f"Porcentaje de infecciones respiratorias principales : {mainresp_SIN_PA:.2f}%")

Porcentaje de infecciones respiratorias principales : 42.86%


In [1635]:
LRTI_SIN_PA= (SIN_PA['LRTI_diagnosticada'] == 1.0).mean() * 100
print(f"Porcentaje de LRTI: {LRTI_SIN_PA:.2f}%")

Porcentaje de LRTI: 22.45%


In [1636]:
COPD_SIN_PA= (SIN_PA['COPD_exacerbations_diagnosticada'] == 1.0).mean() * 100
print(f"Porcentaje de exacerbaciones de COPD: {COPD_SIN_PA:.2f}%")

Porcentaje de exacerbaciones de COPD: 32.65%


In [1637]:
fibro_SIN_PA= (SIN_PA['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0).mean() * 100
print(f"Porcentaje de fibrosis pulmonar y fibrotorax: {fibro_SIN_PA:.2f}%")

Porcentaje de fibrosis pulmonar y fibrotorax: 0.00%


#### Pvalor

##### Infecciones respiratorias principales 

In [1638]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['Main_respiratory_infections_diagnosticada'] == 1.0, CON_PA_MECVV['Main_respiratory_infections_diagnosticada'] == 1.0, equal_var=False)
print("P-valor entre CON_PA y CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA y CON_PA_MECVV: 0.6751021758944709


In [1639]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['Main_respiratory_infections_diagnosticada'] == 1.0, SIN_PA['Main_respiratory_infections_diagnosticada'] == 1.0, equal_var=False)
print("P-valor entre CON_PA y SIN_PA:", p_val_13)

P-valor entre CON_PA y SIN_PA: 0.9068187447260653


In [1640]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['Main_respiratory_infections_diagnosticada'] == 1.0, SIN_PA['Main_respiratory_infections_diagnosticada'] == 1.0, equal_var=False)
print("P-valor entre CON_PA_MECVV y SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV y SIN_PA: 0.7002620618397895


##### LRTI

In [1641]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['LRTI_diagnosticada'] == 1.0, CON_PA_MECVV['LRTI_diagnosticada'] == 1.0, equal_var=False)
print("P-valor entre CON_PA y CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA y CON_PA_MECVV: 0.8874457485314164


In [1642]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['LRTI_diagnosticada'] == 1.0, SIN_PA['LRTI_diagnosticada'] == 1.0, equal_var=False)
print("P-valor entre CON_PA y SIN_PA:", p_val_13)

P-valor entre CON_PA y SIN_PA: 0.6732221433079675


In [1643]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['LRTI_diagnosticada'] == 1.0, SIN_PA['LRTI_diagnosticada'] == 1.0, equal_var=False)
print("P-valor entre CON_PA_MECVV y SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV y SIN_PA: 0.869649462749599


##### Exacerbaciones de COPD

In [1644]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['COPD_exacerbations_diagnosticada'] == 1.0, CON_PA_MECVV['COPD_exacerbations_diagnosticada'] == 1.0, equal_var=False)
print("P-valor entre CON_PA y CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA y CON_PA_MECVV: 0.9757023199212823


In [1645]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['COPD_exacerbations_diagnosticada'] == 1.0, SIN_PA['COPD_exacerbations_diagnosticada'] == 1.0, equal_var=False)
print("P-valor entre CON_PA y SIN_PA:", p_val_13)

P-valor entre CON_PA y SIN_PA: 0.8085270064624923


In [1646]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['COPD_exacerbations_diagnosticada'] == 1.0, SIN_PA['COPD_exacerbations_diagnosticada'] == 1.0, equal_var=False)
print("P-valor entre CON_PA_MECVV y SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV y SIN_PA: 0.8763124168494365


##### Fibrosis pulmonar y fibrotorax

In [1647]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0, CON_PA_MECVV['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0, equal_var=False)
print("P-valor entre CON_PA y CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA y CON_PA_MECVV: nan


In [1648]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0, SIN_PA['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0, equal_var=False)
print("P-valor entre CON_PA y CON_PA_MECVV:", p_val_13)

P-valor entre CON_PA y CON_PA_MECVV: nan


In [1649]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0, SIN_PA['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0, equal_var=False)
print("P-valor entre CON_PA_MECVV y SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV y SIN_PA: nan


## Resultados de laboratorio

Las columnas generadas de estos resultados de laboratorio contienen valores de tipo 'object' y hay que pasarlos previamente a 'float' para poder realizar las operaciones correspondientes.

### Albúmina

In [1650]:
CON_PA.loc[:, 'albumina'] = pd.to_numeric(CON_PA['albumina'], errors='coerce')

promedio_alb = CON_PA['albumina'].mean()
desviacion_estandar_alb = CON_PA['albumina'].std()

resultado = f"{promedio_alb:.2f} ± {desviacion_estandar_alb:.2f}"
print("Promedio ± Desviación Estándar de la albumina:", resultado)

Promedio ± Desviación Estándar de la albumina: 3.34 ± 0.38


In [1651]:
CON_PA_MECVV.loc[:, 'albumina'] = pd.to_numeric(CON_PA_MECVV['albumina'], errors='coerce')

promedio_alb = CON_PA_MECVV['albumina'].mean()
desviacion_estandar_alb = CON_PA_MECVV['albumina'].std()

resultado = f"{promedio_alb:.2f} ± {desviacion_estandar_alb:.2f}"
print("Promedio ± Desviación Estándar de la albumina:", resultado)

Promedio ± Desviación Estándar de la albumina: 3.47 ± 0.55


In [1652]:
SIN_PA.loc[:, 'albumina'] = pd.to_numeric(SIN_PA['albumina'], errors='coerce')

promedio_alb = SIN_PA['albumina'].mean()
desviacion_estandar_alb = SIN_PA['albumina'].std()

resultado = f"{promedio_alb:.2f} ± {desviacion_estandar_alb:.2f}"
print("Promedio ± Desviación Estándar de la albumina:", resultado)

Promedio ± Desviación Estándar de la albumina: 3.58 ± 0.64


#### Pvalor

In [1653]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['albumina'], CON_PA_MECVV['albumina'], equal_var=False)
print("P-valor entre CON_PA y CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA y CON_PA_MECVV: nan


In [1654]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['albumina'], SIN_PA['albumina'], equal_var=False)
print("P-valor entre CON_PA y SIN_PA:", p_val_13)

P-valor entre CON_PA y SIN_PA: nan


In [1655]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['albumina'], SIN_PA['albumina'], equal_var=False)
print("P-valor entre CON_PA_MECVV y SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV y SIN_PA: nan


### Proteïnas totales

In [1656]:
CON_PA.loc[:, 'proteinas totales'] = pd.to_numeric(CON_PA['proteinas totales'], errors='coerce')

promedio_prot = CON_PA['proteinas totales'].mean()
desviacion_estandar_prot = CON_PA['proteinas totales'].std()

resultado = f"{promedio_prot:.2f} ± {desviacion_estandar_prot:.2f}"
print("Promedio ± Desviación Estándar de las proteinas totales:", resultado)

Promedio ± Desviación Estándar de las proteinas totales: 6.03 ± 0.73


In [1657]:
CON_PA_MECVV.loc[:, 'proteinas totales'] = pd.to_numeric(CON_PA_MECVV['proteinas totales'], errors='coerce')

promedio_prot = CON_PA_MECVV['proteinas totales'].mean()
desviacion_estandar_prot = CON_PA_MECVV['proteinas totales'].std()

resultado = f"{promedio_prot:.2f} ± {desviacion_estandar_prot:.2f}"
print("Promedio ± Desviación Estándar de las proteinas totales:", resultado)

Promedio ± Desviación Estándar de las proteinas totales: 6.47 ± 0.76


In [1658]:
SIN_PA.loc[:, 'proteinas totales'] = pd.to_numeric(SIN_PA['proteinas totales'], errors='coerce')

promedio_prot = SIN_PA['proteinas totales'].mean()
desviacion_estandar_prot = SIN_PA['proteinas totales'].std()

resultado = f"{promedio_prot:.2f} ± {desviacion_estandar_prot:.2f}"
print("Promedio ± Desviación Estándar de las proteinas totales:", resultado)

Promedio ± Desviación Estándar de las proteinas totales: 6.12 ± 0.72


#### Pvalor

In [1659]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['proteinas totales'], CON_PA_MECVV['proteinas totales'], equal_var=False)
print("P-valor entre CON_PA y CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA y CON_PA_MECVV: nan


In [1660]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['proteinas totales'], SIN_PA['proteinas totales'], equal_var=False)
print("P-valor entre CON_PA y SIN_PA:", p_val_13)

P-valor entre CON_PA y SIN_PA: nan


In [1661]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['proteinas totales'], SIN_PA['proteinas totales'], equal_var=False)
print("P-valor entre CON_PA_MECVV y SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV y SIN_PA: nan


### Hemoglobina

In [1662]:
CON_PA.loc[:, 'Hb'] = pd.to_numeric(CON_PA['Hb'], errors='coerce')

promedio_hb = CON_PA['Hb'].mean()
desviacion_estandar_hb = CON_PA['Hb'].std()

resultado = f"{promedio_hb:.2f} ± {desviacion_estandar_hb:.2f}"
print("Promedio ± Desviación Estándar de la hemoglobina:", resultado)

Promedio ± Desviación Estándar de la hemoglobina: 12.34 ± 1.77


In [1663]:
CON_PA_MECVV.loc[:, 'Hb'] = pd.to_numeric(CON_PA_MECVV['Hb'], errors='coerce')

promedio_hb = CON_PA_MECVV['Hb'].mean()
desviacion_estandar_hb = CON_PA_MECVV['Hb'].std()

resultado = f"{promedio_hb:.2f} ± {desviacion_estandar_hb:.2f}"
print("Promedio ± Desviación Estándar de la hemoglobina:", resultado)

Promedio ± Desviación Estándar de la hemoglobina: 13.22 ± 1.40


In [1664]:
SIN_PA.loc[:, 'Hb'] = pd.to_numeric(SIN_PA['Hb'], errors='coerce')

promedio_hb = SIN_PA['Hb'].mean()
desviacion_estandar_hb = SIN_PA['Hb'].std()

resultado = f"{promedio_hb:.2f} ± {desviacion_estandar_hb:.2f}"
print("Promedio ± Desviación Estándar de la hemoglobina:", resultado)

Promedio ± Desviación Estándar de la hemoglobina: 13.22 ± 2.16


##### Pvalor

In [1665]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['Hb'], CON_PA_MECVV['Hb'], equal_var=False)
print("P-valor entre CON_PA y CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA y CON_PA_MECVV: nan


In [1666]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['Hb'], SIN_PA['Hb'], equal_var=False)
print("P-valor entre CON_PA y SIN_PA:", p_val_13)

P-valor entre CON_PA y SIN_PA: nan


In [1667]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['Hb'], SIN_PA['Hb'], equal_var=False)
print("P-valor entre CON_PA_MECVV y SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV y SIN_PA: nan


### Colesterol 

In [1668]:
CON_PA.loc[:, 'colesterol total'] = pd.to_numeric(CON_PA['colesterol total'], errors='coerce')

promedio_col = CON_PA['colesterol total'].mean()
desviacion_estandar_col = CON_PA['colesterol total'].std()

resultado = f"{promedio_col:.2f} ± {desviacion_estandar_col:.2f}"
print("Promedio ± Desviación Estándar del colesterol total:", resultado)

Promedio ± Desviación Estándar del colesterol total: 132.10 ± 34.83


In [1669]:
CON_PA_MECVV.loc[:, 'colesterol total'] = pd.to_numeric(CON_PA_MECVV['colesterol total'], errors='coerce')

promedio_col = CON_PA_MECVV['colesterol total'].mean()
desviacion_estandar_col = CON_PA_MECVV['colesterol total'].std()

resultado = f"{promedio_col:.2f} ± {desviacion_estandar_col:.2f}"
print("Promedio ± Desviación Estándar del colesterol total:", resultado)

Promedio ± Desviación Estándar del colesterol total: 111.90 ± 28.37


In [1670]:
SIN_PA.loc[:, 'colesterol total'] = pd.to_numeric(SIN_PA['colesterol total'], errors='coerce')

promedio_col = SIN_PA['colesterol total'].mean()
desviacion_estandar_col = SIN_PA['colesterol total'].std()

resultado = f"{promedio_col:.2f} ± {desviacion_estandar_col:.2f}"
print("Promedio ± Desviación Estándar del colesterol total:", resultado)

Promedio ± Desviación Estándar del colesterol total: 145.27 ± 40.45


#### Pvalor

In [1671]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['colesterol total'], CON_PA_MECVV['colesterol total'], equal_var=False)
print("P-valor entre CON_PA y CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA y CON_PA_MECVV: nan


In [1672]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['colesterol total'], SIN_PA['colesterol total'], equal_var=False)
print("P-valor entre CON_PA y SIN_PA:", p_val_13)

P-valor entre CON_PA y SIN_PA: nan


In [1673]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['colesterol total'], SIN_PA['colesterol total'], equal_var=False)
print("P-valor entre CON_PA_MECVV y SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV y SIN_PA: nan


### Leucócitos

In [1674]:
CON_PA.loc[:, 'leucos'] = pd.to_numeric(CON_PA['leucos'], errors='coerce')

promedio_leu = CON_PA['leucos'].mean()
desviacion_estandar_leu = CON_PA['leucos'].std()

resultado = f"{promedio_leu:.2f} ± {desviacion_estandar_leu:.2f}"
print("Promedio ± Desviación Estándar de los leucocitos:", resultado)

Promedio ± Desviación Estándar de los leucocitos: 12.69 ± 4.88


In [1675]:
CON_PA_MECVV.loc[:, 'leucos'] = pd.to_numeric(CON_PA_MECVV['leucos'], errors='coerce')

promedio_leu = CON_PA_MECVV['leucos'].mean()
desviacion_estandar_leu = CON_PA_MECVV['leucos'].std()

resultado = f"{promedio_leu:.2f} ± {desviacion_estandar_leu:.2f}"
print("Promedio ± Desviación Estándar de los leucocitos:", resultado)

Promedio ± Desviación Estándar de los leucocitos: 13.08 ± 6.83


In [1676]:
SIN_PA.loc[:, 'leucos'] = pd.to_numeric(SIN_PA['leucos'], errors='coerce')

promedio_leu = SIN_PA['leucos'].mean()
desviacion_estandar_leu = SIN_PA['leucos'].std()

resultado = f"{promedio_leu:.2f} ± {desviacion_estandar_leu:.2f}"
print("Promedio ± Desviación Estándar de los leucocitos:", resultado)

Promedio ± Desviación Estándar de los leucocitos: 11.52 ± 6.52


#### Pvalor 

In [1677]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['leucos'], CON_PA_MECVV['leucos'], equal_var=False)
print("P-valor entre CON_PA y CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA y CON_PA_MECVV: nan


In [1678]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['leucos'], SIN_PA['leucos'], equal_var=False)
print("P-valor entre CON_PA y SIN_PA:", p_val_13)

P-valor entre CON_PA y SIN_PA: nan


In [1679]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['leucos'], SIN_PA['leucos'], equal_var=False)
print("P-valor entre CON_PA_MECVV y SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV y SIN_PA: nan


### Linfocitos %

In [1680]:
CON_PA.loc[:, 'limfos'] = pd.to_numeric(CON_PA['limfos'], errors='coerce')

promedio_limf = CON_PA['limfos'].mean()
desviacion_estandar_limf = CON_PA['limfos'].std()

resultado = f"{promedio_limf:.2f} ± {desviacion_estandar_limf:.2f}"
print("Promedio ± Desviación Estándar de los limfocitos %:", resultado)

Promedio ± Desviación Estándar de los limfocitos %: 16.70 ± 12.29


In [1681]:
CON_PA_MECVV.loc[:, 'limfos'] = pd.to_numeric(CON_PA_MECVV['limfos'], errors='coerce')

promedio_limf = CON_PA_MECVV['limfos'].mean()
desviacion_estandar_limf = CON_PA_MECVV['limfos'].std()

resultado = f"{promedio_limf:.2f} ± {desviacion_estandar_limf:.2f}"
print("Promedio ± Desviación Estándar de los limfocitos %:", resultado)

Promedio ± Desviación Estándar de los limfocitos %: 17.20 ± 9.35


In [1682]:
SIN_PA.loc[:, 'limfos'] = pd.to_numeric(SIN_PA['limfos'], errors='coerce')

promedio_limf = SIN_PA['limfos'].mean()
desviacion_estandar_limf = SIN_PA['limfos'].std()

resultado = f"{promedio_limf:.2f} ± {desviacion_estandar_limf:.2f}"
print("Promedio ± Desviación Estándar de los limfocitos %:", resultado)

Promedio ± Desviación Estándar de los limfocitos %: 19.50 ± 11.57


#### Pvalor

In [1683]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['limfos'], CON_PA_MECVV['limfos'], equal_var=False)
print("P-valor entre CON_PA y CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA y CON_PA_MECVV: nan


In [1684]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['limfos'], SIN_PA['limfos'], equal_var=False)
print("P-valor entre CON_PA y SIN_PA:", p_val_13)

P-valor entre CON_PA y SIN_PA: nan


In [1685]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['limfos'], SIN_PA['limfos'], equal_var=False)
print("P-valor entre CON_PA_MECVV y SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV y SIN_PA: nan


### Proteina C Reactiva

In [1686]:
CON_PA.loc[:, 'prot C react'] = pd.to_numeric(CON_PA['prot C react'], errors='coerce')

promedio_protcr = CON_PA['prot C react'].mean()
desviacion_estandar_protcr = CON_PA['prot C react'].std()

resultado = f"{promedio_protcr:.2f} ± {desviacion_estandar_protcr:.2f}"
print("Promedio ± Desviación Estándar de la proteina C reactiva:", resultado)

Promedio ± Desviación Estándar de la proteina C reactiva: 8.90 ± 5.47


In [1687]:
CON_PA_MECVV.loc[:, 'prot C react'] = pd.to_numeric(CON_PA_MECVV['prot C react'], errors='coerce')

promedio_protcr = CON_PA_MECVV['prot C react'].mean()
desviacion_estandar_protcr = CON_PA_MECVV['prot C react'].std()

resultado = f"{promedio_protcr:.2f} ± {desviacion_estandar_protcr:.2f}"
print("Promedio ± Desviación Estándar de la proteina C reactiva:", resultado)

Promedio ± Desviación Estándar de la proteina C reactiva: 8.88 ± 3.63


In [1688]:
SIN_PA.loc[:, 'prot C react'] = pd.to_numeric(SIN_PA['prot C react'], errors='coerce')

promedio_protcr = SIN_PA['prot C react'].mean()
desviacion_estandar_protcr = SIN_PA['prot C react'].std()

resultado = f"{promedio_protcr:.2f} ± {desviacion_estandar_protcr:.2f}"
print("Promedio ± Desviación Estándar de la proteina C reactiva:", resultado)

Promedio ± Desviación Estándar de la proteina C reactiva: 6.16 ± 6.19


#### Pvalor

In [1689]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['prot C react'], CON_PA_MECVV['prot C react'], equal_var=False)
print("P-valor entre CON_PA y CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA y CON_PA_MECVV: nan


In [1690]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['prot C react'], SIN_PA['prot C react'], equal_var=False)
print("P-valor entre CON_PA y SIN_PA:", p_val_13)

P-valor entre CON_PA y SIN_PA: nan


In [1691]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['prot C react'], SIN_PA['prot C react'], equal_var=False)
print("P-valor entre CON_PA_MECVV y SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV y SIN_PA: nan


### Urea 

In [1692]:
CON_PA.loc[:, 'urea'] = pd.to_numeric(CON_PA['urea'], errors='coerce')

promedio_urea = CON_PA['urea'].mean()
desviacion_estandar_urea = CON_PA['urea'].std()

resultado = f"{promedio_urea:.2f} ± {desviacion_estandar_urea:.2f}"
print("Promedio ± Desviación Estándar de la urea:", resultado)

Promedio ± Desviación Estándar de la urea: 62.17 ± 36.37


In [1693]:
CON_PA_MECVV.loc[:, 'urea'] = pd.to_numeric(CON_PA_MECVV['urea'], errors='coerce')

promedio_urea = CON_PA_MECVV['urea'].mean()
desviacion_estandar_urea = CON_PA_MECVV['urea'].std()

resultado = f"{promedio_urea:.2f} ± {desviacion_estandar_urea:.2f}"
print("Promedio ± Desviación Estándar de la urea:", resultado)

Promedio ± Desviación Estándar de la urea: 62.37 ± 37.72


In [1694]:
SIN_PA.loc[:, 'urea'] = pd.to_numeric(SIN_PA['urea'], errors='coerce')

promedio_urea = SIN_PA['urea'].mean()
desviacion_estandar_urea = SIN_PA['urea'].std()

resultado = f"{promedio_urea:.2f} ± {desviacion_estandar_urea:.2f}"
print("Promedio ± Desviación Estándar de la urea:", resultado)

Promedio ± Desviación Estándar de la urea: 55.03 ± 30.66


#### Pvalor

In [1695]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['urea'], CON_PA_MECVV['urea'], equal_var=False)
print("P-valor entre CON_PA y CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA y CON_PA_MECVV: nan


In [1696]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['urea'], SIN_PA['urea'], equal_var=False)
print("P-valor entre CON_PA y SIN_PA:", p_val_13)

P-valor entre CON_PA y SIN_PA: nan


In [1697]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['urea'], SIN_PA['urea'], equal_var=False)
print("P-valor entre CON_PA_MECVV y SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV y SIN_PA: nan


### Filtrado glomerular CDK-EPI

In [1698]:
CON_PA.loc[:, 'FGE CDK-EPI'] = pd.to_numeric(CON_PA['FGE CDK-EPI'], errors='coerce')

promedio_fg = CON_PA['FGE CDK-EPI'].mean()
desviacion_estandar_fg = CON_PA['FGE CDK-EPI'].std()

resultado = f"{promedio_fg:.2f} ± {desviacion_estandar_fg:.2f}"
print("Promedio ± Desviación Estándar de la filtracion glomerular CDK-EPI:", resultado)

Promedio ± Desviación Estándar de la filtracion glomerular CDK-EPI: 56.00 ± 12.65


In [1699]:
CON_PA_MECVV.loc[:, 'FGE CDK-EPI'] = pd.to_numeric(CON_PA_MECVV['FGE CDK-EPI'], errors='coerce')

promedio_fg = CON_PA_MECVV['FGE CDK-EPI'].mean()
desviacion_estandar_fg = CON_PA_MECVV['FGE CDK-EPI'].std()

resultado = f"{promedio_fg:.2f} ± {desviacion_estandar_fg:.2f}"
print("Promedio ± Desviación Estándar de la filtracion glomerular CDK-EPI:", resultado)

Promedio ± Desviación Estándar de la filtracion glomerular CDK-EPI: 54.67 ± 13.32


In [1700]:
SIN_PA.loc[:, 'FGE CDK-EPI'] = pd.to_numeric(SIN_PA['FGE CDK-EPI'], errors='coerce')

promedio_fg = SIN_PA['FGE CDK-EPI'].mean()
desviacion_estandar_fg = SIN_PA['FGE CDK-EPI'].std()

resultado = f"{promedio_fg:.2f} ± {desviacion_estandar_fg:.2f}"
print("Promedio ± Desviación Estándar de la filtracion glomerular CDK-EPI:", resultado)

Promedio ± Desviación Estándar de la filtracion glomerular CDK-EPI: 58.56 ± 18.40


#### Pvalor

In [1701]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['FGE CDK-EPI'], CON_PA_MECVV['FGE CDK-EPI'], equal_var=False)
print("P-valor entre CON_PA y CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA y CON_PA_MECVV: nan


In [1702]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['FGE CDK-EPI'], SIN_PA['FGE CDK-EPI'], equal_var=False)
print("P-valor entre CON_PA y CON_PA_MECVV:", p_val_13)

P-valor entre CON_PA y CON_PA_MECVV: nan


In [1703]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['FGE CDK-EPI'], SIN_PA['FGE CDK-EPI'], equal_var=False)
print("P-valor entre CON_PA_MECVV y SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV y SIN_PA: nan


### Filtrado glomerular MDRD

In [1704]:
CON_PA.loc[:, 'FGE MDRD'] = pd.to_numeric(CON_PA['FGE MDRD'], errors='coerce')

promedio_fgmd = CON_PA['FGE MDRD'].mean()
desviacion_estandar_fgmd = CON_PA['FGE MDRD'].std()

resultado = f"{promedio_fgmd:.2f} ± {desviacion_estandar_fgmd:.2f}"
print("Promedio ± Desviación Estándar de la filtracion glomerular MDRD:", resultado)

Promedio ± Desviación Estándar de la filtracion glomerular MDRD: 51.50 ± 0.71


In [1705]:
CON_PA_MECVV.loc[:, 'FGE MDRD'] = pd.to_numeric(CON_PA_MECVV['FGE MDRD'], errors='coerce')

promedio_fgmd = CON_PA_MECVV['FGE MDRD'].mean()
desviacion_estandar_fgmd = CON_PA_MECVV['FGE MDRD'].std()

resultado = f"{promedio_fgmd:.2f} ± {desviacion_estandar_fgmd:.2f}"
print("Promedio ± Desviación Estándar de la filtracion glomerular MDRD:", resultado)

Promedio ± Desviación Estándar de la filtracion glomerular MDRD: nan ± nan


In [1706]:
SIN_PA.loc[:, 'FGE MDRD'] = pd.to_numeric(SIN_PA['FGE MDRD'], errors='coerce')

promedio_fgmd = SIN_PA['FGE MDRD'].mean()
desviacion_estandar_fgmd = SIN_PA['FGE MDRD'].std()

resultado = f"{promedio_fgmd:.2f} ± {desviacion_estandar_fgmd:.2f}"
print("Promedio ± Desviación Estándar de la filtracion glomerular MDRD:", resultado)

Promedio ± Desviación Estándar de la filtracion glomerular MDRD: 37.67 ± 17.43


#### Pvalor

In [1707]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['FGE MDRD'], CON_PA_MECVV['FGE MDRD'], equal_var=False)
print("P-valor entre CON_PA y CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA y CON_PA_MECVV: nan


In [1708]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['FGE MDRD'], SIN_PA['FGE MDRD'], equal_var=False)
print("P-valor entre CON_PA y CON_PA_MECVV:", p_val_13)

P-valor entre CON_PA y CON_PA_MECVV: nan


In [1709]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['FGE MDRD'], SIN_PA['FGE MDRD'], equal_var=False)
print("P-valor entre CON_PA_MECVV y CON_PA_MECVV:", p_val_23)

P-valor entre CON_PA_MECVV y CON_PA_MECVV: nan
